In [22]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
import numpy as np
import joblib


In [15]:
model = joblib.load('../models/best_model.pkl')

In [16]:
current_data_file_path = '../data/processed/data_final.csv'  # Update the path to your file
data = pd.read_csv(current_data_file_path)
display(data.head())

,Year,Country,gold_medal,silver_medal,bronze_medal,total_medal,type_olympic,host_flag,Country_,gold_medal_sum,...,primary_enrollment_ratio,secondary_enrollment_ratio,tertiary_enrollment_ratio,internet_penetration,mobile_penetration,health_expenditure,Income Group_Low income,Income Group_Lower middle income,Income Group_Upper middle income,host_flag_1
0,2022.0,AUS,1.0,2.0,1.0,4.0,w,0.0,AUS,172.0,...,99.132988,133.277267,106.240761,96.240000,107.031203,10.543639,0.0,0.0,0.0,0.0
1,2022.0,AUT,7.0,7.0,4.0,18.0,w,0.0,AUT,92.0,...,100.775627,101.458000,93.940071,93.614091,123.434807,12.100000,0.0,0.0,0.0,0.0
2,2022.0,BEL,1.0,0.0,1.0,2.0,w,0.0,BEL,45.0,...,101.821136,143.163513,82.688202,94.007831,101.870773,11.042908,0.0,0.0,0.0,0.0
3,2022.0,BLR,0.0,2.0,0.0,2.0,w,0.0,BLR,23.0,...,94.711967,94.535896,70.867569,89.507331,123.447255,6.570000,0.0,0.0,1.0,0.0
4,2022.0,CAN,4.0,8.0,14.0,26.0,w,0.0,CAN,150.0,...,96.293083,109.334312,77.802292,92.834017,91.230625,11.154714,0.0,0.0,0.0,0.0


In [17]:
usa = data[data['Country'] == 'USA']
usa

,Year,Country,gold_medal,silver_medal,bronze_medal,total_medal,type_olympic,host_flag,Country_,gold_medal_sum,...,primary_enrollment_ratio,secondary_enrollment_ratio,tertiary_enrollment_ratio,internet_penetration,mobile_penetration,health_expenditure,Income Group_Low income,Income Group_Lower middle income,Income Group_Upper middle income,host_flag_1
27,2022.0,USA,8.0,10.0,7.0,25.0,w,0.0,USA,1178.0,...,96.971733,97.473488,79.361931,91.753208,110.166472,16.571152,0.0,0.0,1.0,0.0
100,2020.0,USA,39.0,41.0,33.0,113.0,s,0.0,USA,1178.0,...,100.305794,100.509819,87.567657,90.620470,104.935375,18.756220,0.0,0.0,0.0,0.0


In [18]:
X = data.loc[data['Country'] != 'USA', model.X_columns]
y = data.loc[data['Country'] != 'USA', 'total_medal']

In [19]:
len(X), len(y)

(118, 118)

In [20]:
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [23]:
cv_results = cross_validate(model, X, y, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

In [24]:
cv_results['test_score']

array([-2.26366973, -2.67687107, -2.58212253, -0.81979611, -1.5908103 ])

In [25]:
mae_mean, mae_std = np.mean(-1 * cv_results['test_score']), np.std(-1 * cv_results['test_score'])
mae_mean, mae_std

(1.9866539485644605, 0.6965586336499013)

In [26]:
X_usa = data.loc[data['Country'] == 'USA', model.X_columns]
y_usa = data.loc[data['Country'] == 'USA', 'total_medal']

In [29]:
usa_pred = model.predict(X_usa)


array([22.2346 , 88.13481], dtype=float32)

In [36]:
print(f'Winter olympic games predictions = {usa_pred[0]}\nSummer olympic games predictions = {usa_pred[1]}')


Winter olympic games predictions = 22.234600067138672
Summer olympic games predictions = 88.13481140136719


In [37]:
y_usa_a = y_usa.values
print(f'Winter olympic games actual = {y_usa_a[0]}\nSummer olympic games actual = {y_usa_a[1]}')


Winter olympic games actual = 25.0
Summer olympic games actual = 113.0


In [33]:
dif = y_usa_a-usa_pred

In [34]:
dif

array([ 2.76539993, 24.8651886 ])